In [1]:
!pip install sacremoses
from transformers import pipeline

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 11.6 MB/s eta 0:00:0000:0100:01


In [2]:
from transformers import BioGptForSequenceClassification, BioGptTokenizer

model_name = "stanford-crfm/BioMedLM"
model = BioGptForSequenceClassification.from_pretrained(model_name, num_labels=3, problem_type="multi_label_classification")
tokenizer = BioGptTokenizer.from_pretrained(model_name)

from transformers import pipeline

classifier = pipeline("sentiment-analysis", model=model_name, tokenizer=model_name)

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at gpt2-medium and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
import torch
import pandas as pd

In [4]:
def get_decision(question):
    inputs = tokenizer(question, return_tensors="pt", max_length=512, truncation=True)

    with torch.no_grad():
        outputs = model(**inputs)
    
    predicted_class_idx = outputs.logits.argmax(dim=-1).item()
    
    return predicted_class_idx

In [5]:
MAX_TOKENS = 512

def truncate_text_to_fit(text, max_tokens=MAX_TOKENS):
    token_count = len(tokenizer.tokenize(text))
    if token_count > max_tokens - 2:  
        truncated_text = " ".join(text.split()[:-1])
        while len(tokenizer.tokenize(truncated_text)) > max_tokens - 2:
            truncated_text = " ".join(truncated_text.split()[:-1])
    else:
        truncated_text = text
        
    return truncated_text

In [6]:
def create_input_text(question, context):
    input_text = "question: "+ question + "context: " + context
    return input_text

In [7]:
def save_to_csv(patient_id,indexes, pubids, results):
    df = pd.DataFrame({
        'Index': indexes,
        'PubID': pubids,
        'Result': results
    })
    df.to_csv(str(patient_id)+'-results.csv', index=False)

In [12]:
patient_histories_path = '/kaggle/input/gpt-patient/Patient_Histories_Cleaned.csv'
patient_histories_df = pd.read_csv(patient_histories_path)

for patient_id, patient_history in patient_histories_df[['Patient ID', 'Patient History']].values:
    patient_number = patient_id.replace("Patient-", "number")
    summarized_abstracts_path = f'/kaggle/input/gpt-patient/summarized_patient_cases_gpt/summarized_patient_cases_gpt/summarized_patient-{patient_number}-articles.csv'
    summarized_abstracts_df = pd.read_csv(summarized_abstracts_path)
    summarized_abstracts_df = summarized_abstracts_df.dropna()
    summarized_abstracts_df['input_text']=  summarized_abstracts_df['Summary'].apply(lambda context: create_input_text(patient_history, context))
    results = []
    pubids = []
    indexes = []
    for idx, row in summarized_abstracts_df.iterrows():
            indexes.append(idx + 1)
            pubids.append(row['ID'])
            results.append(classifier(truncate_text_to_fit(row['input_text'])))

    if results:
            save_to_csv(patient_id,indexes, pubids, results)
        

In [13]:
import pandas as pd
import json
import os

label_mapping = {
    "LABEL_0": "yes",
    "LABEL_1": "no",
    "LABEL_2": "maybe"
}

def map_label(result_str):
    try:
        result_str = result_str.replace("'", '"')
        result = json.loads(result_str)
        label = result[0]['label']
        return label_mapping.get(label, "Unknown")
    except:
        return "Error in processing"

# Loop through each file
for i in range(1, 17):
    file_name = f"/kaggle/working/Patient-{i}-results.csv"
    
    try:
        # Read the file
        decisions = pd.read_csv(file_name)
        
        # Apply the map_label function
        decisions['Result'] = decisions['Result'].apply(map_label)
        
        # Save the file with a new suffix
        new_file_name = f"/kaggle/working/Patient-{i}-results-converted.csv"
        decisions.to_csv(new_file_name, index=False)
        
        # Delete the original file
        os.remove(file_name)

        print(f"Processed, saved, and deleted: {new_file_name}")

    except FileNotFoundError:
        print(f"File not found: {file_name}")

Processed, saved, and deleted: /kaggle/working/Patient-1-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-2-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-3-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-4-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-5-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-6-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-7-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-8-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-9-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-10-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-11-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Patient-12-results-converted.csv
Processed, saved, and deleted: /kaggle/working/Pa